# 🎓 Aula 2: Engenharia de Dados para IA e Compliance - RAG

## Performance – IA para Devs | Desenvolvimento e Orquestração de Agentes de IA para o Setor Financeiro

---

### 📋 Objetivos desta Aula

Ao final desta aula, você será capaz de:

1. **Diferenciar** Chat genérico de Extração estruturada
2. **Implementar RAG** (Retrieval-Augmented Generation) para auditoria e conformidade bancária
3. **Dominar Function Calling** para estruturação de saídas (Output Parsers)
4. **Criar pipelines de ingestão** de documentos PDF
5. **Desenvolver o projeto "O Auditor de Contratos"**: Extração automática de dados validados

---

### 🛠️ Stack Tecnológico

- **Python 3.10+**
- **LangChain Core** - Framework para RAG
- **ChromaDB** - Vector Store para embeddings
- **PyPDF** - Extração de texto de PDFs
- **Pydantic** - Validação de schemas de saída
- **Modelos de Embedding** - OpenAI Embeddings

---

### 📚 Pré-requisitos

- Ter completado a **Aula 1** (Agentes Cognitivos)
- Entendimento básico de **vetores e similaridade**

---

## 🚀 Parte 0: Configuração do Ambiente

Vamos instalar as dependências adicionais necessárias para esta aula.

In [1]:
# Instalação das dependências necessárias para RAG
# Execute esta célula apenas uma vez

%pip install langchain langchain-openai langchain-community langchain-chroma --quiet
%pip install chromadb pypdf pydantic python-dotenv --quiet
%pip install tiktoken --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Configuração das variáveis de ambiente

import os
from dotenv import load_dotenv

# Carrega variáveis do arquivo .env (se existir)
load_dotenv()

# Configure sua chave de API da OpenAI
if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = "sua-chave-aqui"  # Substitua pela sua chave

print("✅ Ambiente configurado com sucesso!")

✅ Ambiente configurado com sucesso!


---

## 📚 Parte 1: Chat Genérico vs Extração Estruturada

### 1.1 O Problema do Chat Genérico

Quando usamos um LLM para **chat genérico**, a saída é **texto livre** - difícil de processar programaticamente.

```
Pergunta: "Analise este contrato e me diga o valor e prazo."

Resposta Genérica:
"O contrato apresenta um valor total de R$ 150.000,00 com prazo de 24 meses...
Além disso, há cláusulas de reajuste anual baseadas no IPCA..."
```

**Problemas:**
- ❌ Como extrair o valor "150000" programaticamente?
- ❌ Como garantir que sempre teremos os mesmos campos?
- ❌ Como validar se os dados estão corretos?

### 1.2 A Solução: Extração Estruturada

Com **extração estruturada**, forçamos o LLM a retornar dados em um **formato predefinido**:

```json
{
    "valor_contrato": 150000.00,
    "moeda": "BRL",
    "prazo_meses": 24,
    "indice_reajuste": "IPCA",
    "score_risco": "MÉDIO"
}
```

**Vantagens:**
- ✅ Dados estruturados e tipados
- ✅ Fácil integração com sistemas
- ✅ Validação automática com Pydantic

---

### 💻 Demonstração: Chat vs Extração Estruturada

In [3]:
# Exemplo de texto de contrato para análise

TEXTO_CONTRATO_EXEMPLO = """
CONTRATO DE EMPRÉSTIMO PESSOAL Nº 2024-00892

PARTES:
CREDOR: Banco XYZ S.A., CNPJ 00.000.000/0001-00
DEVEDOR: Carlos Eduardo Mendes, CPF 123.456.789-00

CLÁUSULA PRIMEIRA - DO OBJETO
O presente contrato tem por objeto a concessão de empréstimo pessoal 
no valor de R$ 75.000,00 (setenta e cinco mil reais).

CLÁUSULA SEGUNDA - DO PRAZO E PAGAMENTO
O empréstimo será pago em 36 (trinta e seis) parcelas mensais e consecutivas,
no valor de R$ 2.847,22 cada, com vencimento todo dia 10 de cada mês.
Taxa de juros: 1,99% ao mês (26,68% ao ano).

CLÁUSULA TERCEIRA - DAS GARANTIAS
O devedor oferece como garantia:
- Alienação fiduciária de veículo: Honda Civic 2022, placa ABC-1234
- Valor de avaliação da garantia: R$ 95.000,00

CLÁUSULA QUARTA - DA INADIMPLÊNCIA
Em caso de atraso no pagamento, incidirá:
- Multa de 2% sobre o valor da parcela
- Juros de mora de 1% ao mês

CLÁUSULA QUINTA - DO VENCIMENTO ANTECIPADO
O contrato vencerá antecipadamente em caso de:
a) Inadimplência superior a 60 dias
b) Falência ou insolvência do devedor
c) Deterioração ou perecimento da garantia

São Paulo, 15 de Janeiro de 2024.

[Assinaturas]
"""

print("📄 TEXTO DO CONTRATO DE EXEMPLO:")
print("=" * 60)
print(TEXTO_CONTRATO_EXEMPLO)

📄 TEXTO DO CONTRATO DE EXEMPLO:

CONTRATO DE EMPRÉSTIMO PESSOAL Nº 2024-00892

PARTES:
CREDOR: Banco XYZ S.A., CNPJ 00.000.000/0001-00
DEVEDOR: Carlos Eduardo Mendes, CPF 123.456.789-00

CLÁUSULA PRIMEIRA - DO OBJETO
O presente contrato tem por objeto a concessão de empréstimo pessoal 
no valor de R$ 75.000,00 (setenta e cinco mil reais).

CLÁUSULA SEGUNDA - DO PRAZO E PAGAMENTO
O empréstimo será pago em 36 (trinta e seis) parcelas mensais e consecutivas,
no valor de R$ 2.847,22 cada, com vencimento todo dia 10 de cada mês.
Taxa de juros: 1,99% ao mês (26,68% ao ano).

CLÁUSULA TERCEIRA - DAS GARANTIAS
O devedor oferece como garantia:
- Alienação fiduciária de veículo: Honda Civic 2022, placa ABC-1234
- Valor de avaliação da garantia: R$ 95.000,00

CLÁUSULA QUARTA - DA INADIMPLÊNCIA
Em caso de atraso no pagamento, incidirá:
- Multa de 2% sobre o valor da parcela
- Juros de mora de 1% ao mês

CLÁUSULA QUINTA - DO VENCIMENTO ANTECIPADO
O contrato vencerá antecipadamente em caso de:
a) In

In [4]:
# ABORDAGEM 1: Chat Genérico (problemático)

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

prompt_generico = ChatPromptTemplate.from_messages([
    ("system", "Você é um analista de contratos. Analise o contrato fornecido."),
    ("human", "Analise este contrato e extraia as informações principais:\n\n{contrato}")
])

chain_generica = prompt_generico | llm

resposta_generica = chain_generica.invoke({"contrato": TEXTO_CONTRATO_EXEMPLO})

print("❌ ABORDAGEM 1: Chat Genérico")
print("=" * 60)
print(resposta_generica.content)
print("\n⚠️ PROBLEMA: Como extrair o valor 75000 deste texto programaticamente?")
print("   O formato é inconsistente e difícil de processar.")

❌ ABORDAGEM 1: Chat Genérico
Aqui estão as informações principais extraídas do contrato de empréstimo pessoal:

**CONTRATO DE EMPRÉSTIMO PESSOAL Nº 2024-00892**

**PARTES:**
- **CREDOR:** Banco XYZ S.A.
  - **CNPJ:** 00.000.000/0001-00
- **DEVEDOR:** Carlos Eduardo Mendes
  - **CPF:** 123.456.789-00

**CLÁUSULA PRIMEIRA - DO OBJETO:**
- **Valor do Empréstimo:** R$ 75.000,00 (setenta e cinco mil reais).

**CLÁUSULA SEGUNDA - DO PRAZO E PAGAMENTO:**
- **Número de Parcelas:** 36 (trinta e seis) parcelas mensais.
- **Valor de Cada Parcela:** R$ 2.847,22.
- **Vencimento:** Todo dia 10 de cada mês.
- **Taxa de Juros:** 1,99% ao mês (26,68% ao ano).

**CLÁUSULA TERCEIRA - DAS GARANTIAS:**
- **Garantia Oferecida:** Alienação fiduciária de veículo.
  - **Veículo:** Honda Civic 2022, placa ABC-1234.
  - **Valor de Avaliação da Garantia:** R$ 95.000,00.

**CLÁUSULA QUARTA - DA INADIMPLÊNCIA:**
- **Multa por Atraso:** 2% sobre o valor da parcela.
- **Juros de Mora:** 1% ao mês.

**CLÁUSULA QUINTA 

### 🔧 Abordagem 2: Extração Estruturada com Pydantic

Agora vamos usar **Pydantic** para definir exatamente o formato de saída que queremos do LLM.

**O que é Pydantic?**
- Uma biblioteca Python para **validação de dados** usando type hints
- Permite definir **schemas** (modelos de dados) com campos tipados
- O LangChain usa esses schemas para "forçar" o LLM a retornar dados estruturados

**Como funciona:**

1. **Definimos classes** que herdam de `BaseModel` com campos tipados
2. Usamos `Field()` para adicionar **descrições** que orientam o LLM
3. Podemos usar `Enum` para restringir valores a opções específicas
4. O método `with_structured_output()` do LLM garante que a resposta siga o schema

**Vantagens:**
- ✅ Saída sempre no formato esperado (JSON válido)
- ✅ Tipos validados automaticamente (int, float, str, etc.)
- ✅ Campos obrigatórios vs opcionais bem definidos
- ✅ Fácil exportação para JSON com `model_dump_json()`

In [5]:
# ABORDAGEM 2: Extração Estruturada com Pydantic

from pydantic import BaseModel, Field
from typing import Optional, List
from enum import Enum

# Definição do schema de saída
class NivelRisco(str, Enum):
    BAIXO = "BAIXO"
    MEDIO = "MÉDIO"
    ALTO = "ALTO"
    MUITO_ALTO = "MUITO_ALTO"

class TipoGarantia(str, Enum):
    ALIENACAO_FIDUCIARIA = "ALIENAÇÃO_FIDUCIÁRIA"
    HIPOTECA = "HIPOTECA"
    AVAL = "AVAL"
    FIANCA = "FIANÇA"
    SEM_GARANTIA = "SEM_GARANTIA"

class Garantia(BaseModel):
    """Informações sobre garantia do contrato."""
    tipo: TipoGarantia = Field(description="Tipo de garantia oferecida")
    descricao: str = Field(description="Descrição da garantia")
    valor_avaliacao: Optional[float] = Field(default=None, description="Valor de avaliação da garantia em reais")

class ContratoExtraido(BaseModel):
    """Schema para extração estruturada de contratos de empréstimo."""
    
    numero_contrato: str = Field(description="Número identificador do contrato")
    nome_devedor: str = Field(description="Nome completo do devedor")
    cpf_devedor: str = Field(description="CPF do devedor")
    
    valor_emprestimo: float = Field(description="Valor total do empréstimo em reais")
    prazo_meses: int = Field(description="Prazo total em meses")
    valor_parcela: float = Field(description="Valor de cada parcela em reais")
    
    taxa_juros_mensal: float = Field(description="Taxa de juros mensal em percentual")
    taxa_juros_anual: float = Field(description="Taxa de juros anual em percentual")
    
    garantias: List[Garantia] = Field(description="Lista de garantias oferecidas")
    
    multa_atraso_percentual: float = Field(description="Percentual de multa por atraso")
    juros_mora_mensal: float = Field(description="Juros de mora mensal em percentual")
    
    data_contrato: str = Field(description="Data de assinatura do contrato")
    
    # Campos calculados/avaliados pelo modelo
    score_risco: NivelRisco = Field(description="Avaliação de risco do contrato")
    justificativa_risco: str = Field(description="Justificativa para o score de risco atribuído")

print("✅ Schema ContratoExtraido definido com sucesso!")
print(f"\n📋 Campos do schema:")
for field_name, field_info in ContratoExtraido.model_fields.items():
    print(f"   • {field_name}: {field_info.annotation.__name__ if hasattr(field_info.annotation, '__name__') else field_info.annotation}")

✅ Schema ContratoExtraido definido com sucesso!

📋 Campos do schema:
   • numero_contrato: str
   • nome_devedor: str
   • cpf_devedor: str
   • valor_emprestimo: float
   • prazo_meses: int
   • valor_parcela: float
   • taxa_juros_mensal: float
   • taxa_juros_anual: float
   • garantias: List
   • multa_atraso_percentual: float
   • juros_mora_mensal: float
   • data_contrato: str
   • score_risco: NivelRisco
   • justificativa_risco: str


In [6]:
# Usando with_structured_output para extração

from langchain_openai import ChatOpenAI

# LLM configurado para saída estruturada
llm_estruturado = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Cria uma versão do LLM que retorna objetos Pydantic
extrator = llm_estruturado.with_structured_output(ContratoExtraido)

# Prompt para extração
prompt_extracao = ChatPromptTemplate.from_messages([
    ("system", """Você é um especialista em análise de contratos financeiros.
Extraia TODAS as informações solicitadas do contrato fornecido.
Seja preciso com valores numéricos e datas.
Avalie o risco considerando: taxa de juros, garantias, prazo e valor.

Critérios de Risco:
- BAIXO: Garantia > 100% do valor, taxa < 2% a.m., prazo < 24 meses
- MÉDIO: Garantia entre 80-100%, taxa 2-3% a.m., prazo 24-48 meses
- ALTO: Garantia < 80% ou taxa > 3% a.m. ou prazo > 48 meses
- MUITO_ALTO: Sem garantia ou múltiplos fatores de risco"""),
    ("human", "Extraia as informações deste contrato:\n\n{contrato}")
])

chain_estruturada = prompt_extracao | extrator

print("🔧 Chain de extração estruturada configurada!")

🔧 Chain de extração estruturada configurada!


In [7]:
# Executando a extração estruturada

contrato_extraido = chain_estruturada.invoke({"contrato": TEXTO_CONTRATO_EXEMPLO})

print("✅ ABORDAGEM 2: Extração Estruturada")
print("=" * 60)
print(f"\n📋 DADOS EXTRAÍDOS DO CONTRATO:")
print(f"\n🔢 Número: {contrato_extraido.numero_contrato}")
print(f"👤 Devedor: {contrato_extraido.nome_devedor} (CPF: {contrato_extraido.cpf_devedor})")
print(f"\n💰 VALORES:")
print(f"   Valor do Empréstimo: R$ {contrato_extraido.valor_emprestimo:,.2f}")
print(f"   Prazo: {contrato_extraido.prazo_meses} meses")
print(f"   Valor da Parcela: R$ {contrato_extraido.valor_parcela:,.2f}")
print(f"\n📊 TAXAS:")
print(f"   Juros Mensal: {contrato_extraido.taxa_juros_mensal}%")
print(f"   Juros Anual: {contrato_extraido.taxa_juros_anual}%")
print(f"\n🛡️ GARANTIAS:")
for g in contrato_extraido.garantias:
    print(f"   • {g.tipo.value}: {g.descricao}")
    if g.valor_avaliacao:
        print(f"     Valor: R$ {g.valor_avaliacao:,.2f}")
print(f"\n⚠️ RISCO:")
print(f"   Score: {contrato_extraido.score_risco.value}")
print(f"   Justificativa: {contrato_extraido.justificativa_risco}")

✅ ABORDAGEM 2: Extração Estruturada

📋 DADOS EXTRAÍDOS DO CONTRATO:

🔢 Número: 2024-00892
👤 Devedor: Carlos Eduardo Mendes (CPF: 123.456.789-00)

💰 VALORES:
   Valor do Empréstimo: R$ 75,000.00
   Prazo: 36 meses
   Valor da Parcela: R$ 2,847.22

📊 TAXAS:
   Juros Mensal: 1.99%
   Juros Anual: 26.68%

🛡️ GARANTIAS:
   • ALIENAÇÃO_FIDUCIÁRIA: Alienação fiduciária de veículo: Honda Civic 2022, placa ABC-1234
     Valor: R$ 95,000.00

⚠️ RISCO:
   Score: ALTO
   Justificativa: Garantia de 126.67% do valor do empréstimo, taxa de juros de 1.99% ao mês e prazo de 36 meses.


In [8]:
# Agora podemos usar os dados programaticamente!

print("\n🎯 USANDO OS DADOS PROGRAMATICAMENTE:")
print("=" * 60)

# Cálculo de cobertura da garantia
valor_total_garantias = sum(g.valor_avaliacao or 0 for g in contrato_extraido.garantias)
cobertura = (valor_total_garantias / contrato_extraido.valor_emprestimo) * 100

print(f"\n📊 ANÁLISE AUTOMÁTICA:")
print(f"   Valor total das garantias: R$ {valor_total_garantias:,.2f}")
print(f"   Cobertura: {cobertura:.1f}%")

# Cálculo do custo total
custo_total = contrato_extraido.valor_parcela * contrato_extraido.prazo_meses
juros_total = custo_total - contrato_extraido.valor_emprestimo

print(f"\n💵 CUSTO TOTAL DO EMPRÉSTIMO:")
print(f"   Total a pagar: R$ {custo_total:,.2f}")
print(f"   Juros totais: R$ {juros_total:,.2f}")
print(f"   Percentual de juros: {(juros_total/contrato_extraido.valor_emprestimo)*100:.1f}%")

# Decisão automática baseada em regras
print(f"\n🤖 DECISÃO AUTOMÁTICA:")
if cobertura >= 100 and contrato_extraido.score_risco in [NivelRisco.BAIXO, NivelRisco.MEDIO]:
    print("   ✅ APROVAR - Garantia adequada e risco aceitável")
elif cobertura >= 80 and contrato_extraido.score_risco == NivelRisco.MEDIO:
    print("   ⚠️ APROVAR COM RESSALVAS - Revisar condições")
else:
    print("   ❌ ENCAMINHAR PARA COMITÊ - Requer análise manual")

# Exportar como JSON
print(f"\n📤 EXPORTAR COMO JSON:")
print(contrato_extraido.model_dump_json(indent=2))


🎯 USANDO OS DADOS PROGRAMATICAMENTE:

📊 ANÁLISE AUTOMÁTICA:
   Valor total das garantias: R$ 95,000.00
   Cobertura: 126.7%

💵 CUSTO TOTAL DO EMPRÉSTIMO:
   Total a pagar: R$ 102,499.92
   Juros totais: R$ 27,499.92
   Percentual de juros: 36.7%

🤖 DECISÃO AUTOMÁTICA:
   ❌ ENCAMINHAR PARA COMITÊ - Requer análise manual

📤 EXPORTAR COMO JSON:
{
  "numero_contrato": "2024-00892",
  "nome_devedor": "Carlos Eduardo Mendes",
  "cpf_devedor": "123.456.789-00",
  "valor_emprestimo": 75000.0,
  "prazo_meses": 36,
  "valor_parcela": 2847.22,
  "taxa_juros_mensal": 1.99,
  "taxa_juros_anual": 26.68,
  "garantias": [
    {
      "tipo": "ALIENAÇÃO_FIDUCIÁRIA",
      "descricao": "Alienação fiduciária de veículo: Honda Civic 2022, placa ABC-1234",
      "valor_avaliacao": 95000.0
    }
  ],
  "multa_atraso_percentual": 2.0,
  "juros_mora_mensal": 1.0,
  "data_contrato": "2024-01-15",
  "score_risco": "ALTO",
  "justificativa_risco": "Garantia de 126.67% do valor do empréstimo, taxa de juros de 1.

---

## 📚 Parte 2: Introdução ao RAG (Retrieval-Augmented Generation)

### 2.1 O Problema: Conhecimento Limitado do LLM

LLMs têm **conhecimento estático** (data de corte do treinamento) e **não conhecem seus documentos internos**.

```
Pergunta: "Qual é a política de crédito do nosso banco para clientes PJ?"
LLM: "Não tenho acesso a documentos internos da sua empresa..."
```

### 2.2 A Solução: RAG

**RAG** combina:
1. **Retrieval (Busca)**: Encontrar documentos relevantes em uma base de conhecimento
2. **Augmented (Aumentado)**: Adicionar esses documentos ao contexto do LLM
3. **Generation (Geração)**: LLM gera resposta baseada no contexto fornecido

```
┌─────────────────────────────────────────────────────────────────────────┐
│                         ARQUITETURA RAG                                  │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   ┌──────────────┐     ┌──────────────────┐     ┌──────────────┐        │
│   │  Documentos  │ ──▶ │  Chunking +      │ ──▶ │ Vector Store │        │
│   │  (PDFs,etc)  │     │  Embedding       │     │  (ChromaDB)  │        │
│   └──────────────┘     └──────────────────┘     └──────────────┘        │
│                                                         │               │
│                                                         ▼               │
│   ┌──────────────┐     ┌──────────────────┐     ┌──────────────┐        │
│   │   Pergunta   │ ──▶ │  Embedding da    │ ──▶ │   Busca      │        │
│   │   do Usuário │     │  Pergunta        │     │   Semântica  │        │
│   └──────────────┘     └──────────────────┘     └──────────────┘        │
│                                                         │               │
│                                                         ▼               │
│   ┌──────────────┐     ┌──────────────────┐     ┌──────────────┐        │
│   │   Resposta   │ ◀── │      LLM         │ ◀── │  Documentos  │        │
│   │   Final      │     │   + Contexto     │     │  Relevantes  │        │
│   └──────────────┘     └──────────────────┘     └──────────────┘        │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

### 2.3 Conceitos Fundamentais

| Conceito | Descrição |
|----------|----------|
| **Embedding** | Representação numérica (vetor) de texto |
| **Vector Store** | Banco de dados especializado em vetores |
| **Chunking** | Divisão de documentos grandes em pedaços menores |
| **Similaridade** | Medida de quão parecidos são dois vetores |

---

In [9]:
# Demonstração visual de Embeddings

from langchain_openai import OpenAIEmbeddings
import numpy as np

# Inicializa o modelo de embeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Textos de exemplo
textos = [
    "Contrato de empréstimo pessoal com garantia",
    "Financiamento de veículo com alienação fiduciária",
    "Receita de bolo de chocolate com cobertura",
    "Política de crédito para pessoa jurídica",
]

# Gera embeddings
vetores = embeddings.embed_documents(textos)

print("🔢 DEMONSTRAÇÃO DE EMBEDDINGS")
print("=" * 60)
print(f"\nDimensão dos vetores: {len(vetores[0])} dimensões")
print(f"\nPrimeiros 10 valores do embedding do texto 1:")
print(f"   {vetores[0][:10]}")

🔢 DEMONSTRAÇÃO DE EMBEDDINGS

Dimensão dos vetores: 1536 dimensões

Primeiros 10 valores do embedding do texto 1:
   [-0.0067872400395572186, 0.03852217271924019, -0.0045932140201330185, 0.03574162721633911, -0.0616740956902504, -0.019106611609458923, -0.053177978843450546, 0.0506291426718235, 0.00847198162227869, -0.03352104872465134]


### 3.3 Similaridade de Cosseno

A **similaridade de cosseno** mede o ângulo entre dois vetores, indicando quão parecidos eles são.

**Fórmula:**
$$\text{similaridade} = \frac{\vec{v_1} \cdot \vec{v_2}}{|\vec{v_1}| \times |\vec{v_2}|}$$

Onde:
- $\vec{v_1} \cdot \vec{v_2}$ = produto escalar dos vetores
- $|\vec{v_1}|$ = norma (magnitude) do vetor 1
- $|\vec{v_2}|$ = norma do vetor 2

**Interpretação:**
- Valor = **1**: Vetores idênticos (mesma direção)
- Valor = **0**: Vetores ortogonais (nenhuma similaridade)
- Valor = **-1**: Vetores opostos (direções completamente diferentes)

**Exemplo prático:**
- "taxa de administração" e "custos de gestão" → similaridade ≈ 0.95 (muito similares!)
- "taxa de administração" e "receita de bolo" → similaridade ≈ 0.10 (muito diferentes)

In [10]:
# Calculando similaridade entre textos

from numpy import dot
from numpy.linalg import norm

def similaridade_cosseno(v1, v2):
    """Calcula a similaridade de cosseno entre dois vetores."""
    return dot(v1, v2) / (norm(v1) * norm(v2))

print("📊 MATRIZ DE SIMILARIDADE")
print("=" * 60)
print("\nTextos:")
for i, t in enumerate(textos):
    print(f"   [{i}] {t}")

print("\nSimilaridade entre textos (0 = nada similar, 1 = idêntico):")
print("\n        [0]    [1]    [2]    [3]")
for i in range(len(textos)):
    linha = f"   [{i}] "
    for j in range(len(textos)):
        sim = similaridade_cosseno(vetores[i], vetores[j])
        linha += f"{sim:.3f}  "
    print(linha)

print("\n💡 Observe:")
print("   - Textos [0], [1] e [3] são sobre finanças → alta similaridade entre si")
print("   - Texto [2] é sobre culinária → baixa similaridade com os demais")

📊 MATRIZ DE SIMILARIDADE

Textos:
   [0] Contrato de empréstimo pessoal com garantia
   [1] Financiamento de veículo com alienação fiduciária
   [2] Receita de bolo de chocolate com cobertura
   [3] Política de crédito para pessoa jurídica

Similaridade entre textos (0 = nada similar, 1 = idêntico):

        [0]    [1]    [2]    [3]
   [0] 1.000  0.505  0.261  0.516  
   [1] 0.505  1.000  0.210  0.505  
   [2] 0.261  0.210  1.000  0.197  
   [3] 0.516  0.505  0.197  1.000  

💡 Observe:
   - Textos [0], [1] e [3] são sobre finanças → alta similaridade entre si
   - Texto [2] é sobre culinária → baixa similaridade com os demais


---

## 📚 Parte 3: Construindo um Sistema RAG Completo

Vamos construir um sistema RAG passo a passo para análise de documentos de compliance bancário.

In [11]:
# Passo 1: Criando documentos de exemplo (simulando PDFs de políticas bancárias)

DOCUMENTOS_COMPLIANCE = [
    {
        "titulo": "Política de Crédito PF - Versão 2024",
        "conteudo": """
POLÍTICA DE CONCESSÃO DE CRÉDITO PARA PESSOA FÍSICA
Documento: POL-CRED-PF-2024 | Versão: 3.2 | Vigência: 01/01/2024

1. CRITÉRIOS DE ELEGIBILIDADE
1.1. Score mínimo de crédito: 650 pontos
1.2. Renda mínima comprovada: R$ 2.500,00
1.3. Tempo mínimo de conta: 6 meses
1.4. Idade: entre 18 e 75 anos
1.5. Sem restrições no SERASA/SPC nos últimos 12 meses

2. LIMITES DE CRÉDITO
2.1. Limite máximo sem garantia: 10x a renda mensal
2.2. Limite máximo com garantia: 30x a renda mensal
2.3. Comprometimento máximo de renda: 35% para crédito pessoal

3. TAXAS DE JUROS
3.1. Crédito pessoal sem garantia: 2,49% a 4,99% ao mês
3.2. Crédito com garantia de veículo: 1,49% a 2,49% ao mês
3.3. Crédito com garantia de imóvel: 0,99% a 1,49% ao mês

4. DOCUMENTAÇÃO NECESSÁRIA
4.1. Documento de identidade (RG ou CNH)
4.2. CPF
4.3. Comprovante de renda dos últimos 3 meses
4.4. Comprovante de residência (últimos 90 dias)
        """
    },
    {
        "titulo": "Política de Crédito PJ - Versão 2024",
        "conteudo": """
POLÍTICA DE CONCESSÃO DE CRÉDITO PARA PESSOA JURÍDICA
Documento: POL-CRED-PJ-2024 | Versão: 2.1 | Vigência: 01/01/2024

1. CRITÉRIOS DE ELEGIBILIDADE
1.1. Tempo mínimo de atividade: 12 meses
1.2. Faturamento mínimo anual: R$ 240.000,00
1.3. Certidões negativas de débitos federais, estaduais e municipais
1.4. Regularidade no CNPJ

2. LIMITES DE CRÉDITO
2.1. Capital de giro: até 50% do faturamento mensal médio
2.2. Financiamento de equipamentos: até 80% do valor do bem
2.3. Antecipação de recebíveis: até 90% do valor das duplicatas

3. GARANTIAS ACEITAS
3.1. Alienação fiduciária de bens móveis
3.2. Hipoteca de imóveis
3.3. Recebíveis (duplicatas, cheques)
3.4. Aval dos sócios

4. TAXAS DE JUROS
4.1. Capital de giro: 1,89% a 3,49% ao mês
4.2. Financiamento de equipamentos: 1,29% a 2,29% ao mês
4.3. Antecipação de recebíveis: 1,49% a 2,49% ao mês
        """
    },
    {
        "titulo": "Política de Prevenção à Lavagem de Dinheiro",
        "conteudo": """
POLÍTICA DE PREVENÇÃO À LAVAGEM DE DINHEIRO E FINANCIAMENTO AO TERRORISMO
Documento: POL-PLD-2024 | Versão: 5.0 | Vigência: 01/01/2024

1. OPERAÇÕES QUE REQUEREM COMUNICAÇÃO AO COAF
1.1. Depósitos em espécie acima de R$ 50.000,00
1.2. Transferências internacionais acima de R$ 10.000,00
1.3. Operações com características atípicas
1.4. Movimentações incompatíveis com o perfil do cliente

2. PROCEDIMENTOS DE KYC (Know Your Customer)
2.1. Atualização cadastral obrigatória a cada 24 meses
2.2. Verificação de PEP (Pessoa Exposta Politicamente)
2.3. Consulta a listas restritivas internacionais
2.4. Análise de origem dos recursos para valores > R$ 100.000,00

3. ALERTAS AUTOMÁTICOS
3.1. Fragmentação de depósitos (structuring)
3.2. Movimentação em horários atípicos
3.3. Múltiplas transferências para mesma conta
3.4. Operações em localidades de alto risco

4. RESPONSABILIDADES
4.1. Gerente de conta: primeira linha de defesa
4.2. Compliance: análise e comunicação ao COAF
4.3. Diretoria: aprovação de operações atípicas
        """
    },
    {
        "titulo": "Manual de Auditoria de Contratos",
        "conteudo": """
MANUAL DE AUDITORIA DE CONTRATOS DE CRÉDITO
Documento: MAN-AUD-2024 | Versão: 1.5 | Vigência: 01/03/2024

1. CHECKLIST DE AUDITORIA
1.1. Verificar assinaturas de todas as partes
1.2. Conferir valores numéricos e por extenso
1.3. Validar taxas dentro dos limites aprovados
1.4. Confirmar registro de garantias em cartório
1.5. Verificar aprovação de alçada competente

2. CRITÉRIOS DE CONFORMIDADE
2.1. Taxa de juros não pode exceder 150% da taxa média BACEN
2.2. Multa de atraso limitada a 2%
2.3. Juros de mora limitados a 1% ao mês
2.4. CET (Custo Efetivo Total) deve estar claramente informado

3. CLASSIFICAÇÃO DE NÃO CONFORMIDADES
3.1. Crítica: Requer correção imediata e reporte à diretoria
3.2. Maior: Requer correção em até 30 dias
3.3. Menor: Requer correção em até 90 dias
3.4. Observação: Melhoria recomendada

4. FREQUÊNCIA DE AUDITORIA
4.1. Contratos > R$ 1.000.000: auditoria 100%
4.2. Contratos R$ 100.000 - R$ 1.000.000: amostragem 30%
4.3. Contratos < R$ 100.000: amostragem 10%
        """
    },
]

print("📚 DOCUMENTOS DE COMPLIANCE CARREGADOS")
print("=" * 60)
for i, doc in enumerate(DOCUMENTOS_COMPLIANCE):
    print(f"\n{i+1}. {doc['titulo']}")
    print(f"   Caracteres: {len(doc['conteudo'])}")

📚 DOCUMENTOS DE COMPLIANCE CARREGADOS

1. Política de Crédito PF - Versão 2024
   Caracteres: 917

2. Política de Crédito PJ - Versão 2024
   Caracteres: 864

3. Política de Prevenção à Lavagem de Dinheiro
   Caracteres: 1032

4. Manual de Auditoria de Contratos
   Caracteres: 1005


In [12]:
# Passo 2: Chunking - Dividindo documentos em pedaços menores

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

# Configuração do text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,         # Tamanho máximo de cada chunk
    chunk_overlap=50,        # Sobreposição entre chunks
    length_function=len,
    separators=["\n\n", "\n", ". ", " ", ""]  # Prioridade de separação
)

# Criar documentos LangChain com metadados
documentos_langchain = []
for doc in DOCUMENTOS_COMPLIANCE:
    chunks = text_splitter.split_text(doc["conteudo"])
    for i, chunk in enumerate(chunks):
        documento = Document(
            page_content=chunk,
            metadata={
                "titulo": doc["titulo"],
                "chunk_id": i,
                "total_chunks": len(chunks)
            }
        )
        documentos_langchain.append(documento)

print("✂️ CHUNKING REALIZADO")
print("=" * 60)
print(f"\nDocumentos originais: {len(DOCUMENTOS_COMPLIANCE)}")
print(f"Total de chunks gerados: {len(documentos_langchain)}")
print(f"\n📄 Exemplo de chunk:")
print(f"   Conteúdo: {documentos_langchain[0].page_content[:200]}...")
print(f"   Metadados: {documentos_langchain[0].metadata}")

✂️ CHUNKING REALIZADO

Documentos originais: 4
Total de chunks gerados: 12

📄 Exemplo de chunk:
   Conteúdo: POLÍTICA DE CONCESSÃO DE CRÉDITO PARA PESSOA FÍSICA
Documento: POL-CRED-PF-2024 | Versão: 3.2 | Vigência: 01/01/2024

1. CRITÉRIOS DE ELEGIBILIDADE
1.1. Score mínimo de crédito: 650 pontos
1.2. Renda ...
   Metadados: {'titulo': 'Política de Crédito PF - Versão 2024', 'chunk_id': 0, 'total_chunks': 3}


In [13]:
# Passo 3: Criando o Vector Store com ChromaDB

from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

# Inicializa o modelo de embeddings
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

# Cria o vector store em memória
vector_store = Chroma.from_documents(
    documents=documentos_langchain,
    embedding=embedding_model,
    collection_name="compliance_docs"
)

print("🗄️ VECTOR STORE CRIADO")
print("=" * 60)
print(f"\nBanco de dados: ChromaDB (em memória)")
print(f"Modelo de embedding: text-embedding-3-small")
print(f"Documentos indexados: {len(documentos_langchain)}")

🗄️ VECTOR STORE CRIADO

Banco de dados: ChromaDB (em memória)
Modelo de embedding: text-embedding-3-small
Documentos indexados: 12


In [14]:
# Passo 4: Testando a busca semântica

print("🔍 TESTE DE BUSCA SEMÂNTICA")
print("=" * 60)

# Pergunta de teste
pergunta = "Qual é o score mínimo para aprovar crédito para pessoa física?"

# Busca os documentos mais relevantes
docs_relevantes = vector_store.similarity_search(pergunta, k=3)

print(f"\n❓ Pergunta: {pergunta}")
print(f"\n📄 Top 3 documentos mais relevantes:")
for i, doc in enumerate(docs_relevantes):
    print(f"\n--- Documento {i+1} ---")
    print(f"Fonte: {doc.metadata['titulo']}")
    print(f"Conteúdo: {doc.page_content[:300]}...")

🔍 TESTE DE BUSCA SEMÂNTICA

❓ Pergunta: Qual é o score mínimo para aprovar crédito para pessoa física?

📄 Top 3 documentos mais relevantes:

--- Documento 1 ---
Fonte: Política de Crédito PF - Versão 2024
Conteúdo: POLÍTICA DE CONCESSÃO DE CRÉDITO PARA PESSOA FÍSICA
Documento: POL-CRED-PF-2024 | Versão: 3.2 | Vigência: 01/01/2024

1. CRITÉRIOS DE ELEGIBILIDADE
1.1. Score mínimo de crédito: 650 pontos
1.2. Renda mínima comprovada: R$ 2.500,00
1.3. Tempo mínimo de conta: 6 meses
1.4. Idade: entre 18 e 75 anos
1....

--- Documento 2 ---
Fonte: Política de Crédito PJ - Versão 2024
Conteúdo: POLÍTICA DE CONCESSÃO DE CRÉDITO PARA PESSOA JURÍDICA
Documento: POL-CRED-PJ-2024 | Versão: 2.1 | Vigência: 01/01/2024

1. CRITÉRIOS DE ELEGIBILIDADE
1.1. Tempo mínimo de atividade: 12 meses
1.2. Faturamento mínimo anual: R$ 240.000,00
1.3. Certidões negativas de débitos federais, estaduais e munici...

--- Documento 3 ---
Fonte: Política de Crédito PF - Versão 2024
Conteúdo: 2. LIMITES DE CRÉDITO
2.1. 

In [15]:
# Passo 5: Criando a Chain RAG completa

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# Configuração do retriever
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 4}  # Retorna os 4 documentos mais relevantes
)

# Prompt para RAG
rag_prompt = ChatPromptTemplate.from_messages([
    ("system", """Você é um especialista em compliance bancário.
Use APENAS as informações fornecidas no contexto para responder.
Se a informação não estiver no contexto, diga que não encontrou nos documentos disponíveis.
Sempre cite o documento fonte da informação.

Contexto dos documentos:
{context}"""),
    ("human", "{question}")
])

# LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Função para formatar documentos
def format_docs(docs):
    formatted = []
    for doc in docs:
        formatted.append(f"[Fonte: {doc.metadata['titulo']}]\n{doc.page_content}")
    return "\n\n---\n\n".join(formatted)

# Chain RAG
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

print("✅ CHAIN RAG CONFIGURADA!")
print("   Retriever: similarity search (k=4)")
print("   LLM: gpt-4o-mini")

✅ CHAIN RAG CONFIGURADA!
   Retriever: similarity search (k=4)
   LLM: gpt-4o-mini


In [16]:
# Testando o RAG com perguntas sobre compliance

perguntas_teste = [
    "Qual é o score mínimo de crédito para pessoa física?",
    "Quais operações devem ser comunicadas ao COAF?",
    "Qual é o limite máximo de comprometimento de renda?",
    "De quanto em quanto tempo deve ser feita atualização cadastral?",
    "Qual a frequência de auditoria para contratos acima de R$ 1 milhão?",
]

print("🤖 SISTEMA RAG - CONSULTA DE COMPLIANCE")
print("=" * 60)

for pergunta in perguntas_teste:
    print(f"\n❓ {pergunta}")
    resposta = rag_chain.invoke(pergunta)
    print(f"\n💬 {resposta}")
    print("\n" + "-" * 60)

🤖 SISTEMA RAG - CONSULTA DE COMPLIANCE

❓ Qual é o score mínimo de crédito para pessoa física?

💬 O score mínimo de crédito para pessoa física é de 650 pontos. (Fonte: Política de Crédito PF - Versão 2024)

------------------------------------------------------------

❓ Quais operações devem ser comunicadas ao COAF?

💬 As operações que devem ser comunicadas ao COAF são:

1. Depósitos em espécie acima de R$ 50.000,00
2. Transferências internacionais acima de R$ 10.000,00
3. Operações com características atípicas
4. Movimentações incompatíveis com o perfil do cliente

(Fonte: Política de Prevenção à Lavagem de Dinheiro)

------------------------------------------------------------

❓ Qual é o limite máximo de comprometimento de renda?

💬 O limite máximo de comprometimento de renda é de 35% para crédito pessoal. (Fonte: Política de Crédito PF - Versão 2024)

------------------------------------------------------------

❓ De quanto em quanto tempo deve ser feita atualização cadastral?

💬 A

---

## 📚 Parte 4: Ingestão de Documentos PDF

Agora vamos aprender a processar documentos PDF reais, que é o formato mais comum em ambientes corporativos.

In [17]:
# Criando um PDF de exemplo para demonstração
# Em produção, você carregaria PDFs reais

import os

# Conteúdo do PDF de exemplo (proposta de crédito)
PROPOSTA_CREDITO_TEXTO = """
PROPOSTA DE CRÉDITO EMPRESARIAL
Protocolo: PROP-2024-00547
Data: 20/01/2024

═══════════════════════════════════════════════════════════════
                    DADOS DO PROPONENTE
═══════════════════════════════════════════════════════════════

Razão Social: Tech Solutions Ltda.
CNPJ: 12.345.678/0001-90
Data de Fundação: 15/03/2019
Faturamento Anual (2023): R$ 3.500.000,00
Número de Funcionários: 45
Setor: Tecnologia da Informação

Sócios:
- Roberto Almeida (60%) - CPF: 111.222.333-44
- Fernanda Costa (40%) - CPF: 555.666.777-88

═══════════════════════════════════════════════════════════════
                    DADOS DA OPERAÇÃO
═══════════════════════════════════════════════════════════════

Tipo de Operação: Capital de Giro
Valor Solicitado: R$ 500.000,00 (quinhentos mil reais)
Prazo Desejado: 24 meses
Finalidade: Expansão da equipe de desenvolvimento e aquisição de equipamentos

═══════════════════════════════════════════════════════════════
                    GARANTIAS OFERECIDAS
═══════════════════════════════════════════════════════════════

1. Alienação Fiduciária de Equipamentos
   - Descrição: Servidores e equipamentos de TI
   - Valor de Avaliação: R$ 180.000,00

2. Recebíveis (Duplicatas)
   - Descrição: Contratos de prestação de serviços
   - Valor: R$ 250.000,00
   - Vencimento médio: 60 dias

3. Aval dos Sócios
   - Roberto Almeida - Patrimônio declarado: R$ 800.000,00
   - Fernanda Costa - Patrimônio declarado: R$ 450.000,00

Total de Garantias: R$ 1.680.000,00
Cobertura: 336% do valor solicitado

═══════════════════════════════════════════════════════════════
                    ANÁLISE FINANCEIRA
═══════════════════════════════════════════════════════════════

Indicadores (últimos 12 meses):
- Margem EBITDA: 18%
- Liquidez Corrente: 1,45
- Endividamento/PL: 0,65
- Cobertura de Juros (EBITDA/Despesas Financeiras): 4,2x

Histórico de Crédito:
- Score Serasa PJ: 850/1000
- Sem protestos ou pendências judiciais
- Cliente do banco desde: 2020
- Operações anteriores: 2 (todas quitadas sem atraso)

═══════════════════════════════════════════════════════════════
                    CONDIÇÕES PROPOSTAS
═══════════════════════════════════════════════════════════════

Taxa de Juros: 1,89% ao mês (25,12% ao ano)
Sistema de Amortização: PRICE
Valor da Parcela: R$ 25.847,33
IOF: R$ 4.750,00
TAC (Taxa de Abertura): R$ 2.500,00
CET (Custo Efetivo Total): 28,45% ao ano

Data de Vencimento: Todo dia 10
Carência: Sem carência

═══════════════════════════════════════════════════════════════
                    PARECER DA ÁREA COMERCIAL
═══════════════════════════════════════════════════════════════

Cliente com bom histórico e relacionamento sólido com o banco.
Empresa em crescimento no setor de tecnologia.
Garantias robustas que cobrem mais de 3x o valor da operação.

Recomendação: FAVORÁVEL À APROVAÇÃO

Gerente Responsável: Patricia Mendonça
Matrícula: 45678
Agência: 0001 - Centro

═══════════════════════════════════════════════════════════════
"""

# Salvar como arquivo de texto (simulando PDF)
os.makedirs("documentos", exist_ok=True)
with open("documentos/proposta_credito.txt", "w", encoding="utf-8") as f:
    f.write(PROPOSTA_CREDITO_TEXTO)

print("📄 PROPOSTA DE CRÉDITO CRIADA")
print("=" * 60)
print(f"Arquivo: documentos/proposta_credito.txt")
print(f"Tamanho: {len(PROPOSTA_CREDITO_TEXTO)} caracteres")

📄 PROPOSTA DE CRÉDITO CRIADA
Arquivo: documentos/proposta_credito.txt
Tamanho: 3002 caracteres


In [18]:
# Criando mais documentos de exemplo

CONTRATO_EXEMPLO_2 = """
PROPOSTA DE FINANCIAMENTO DE VEÍCULO
Protocolo: FIN-VEI-2024-01234
Data: 22/01/2024

═══════════════════════════════════════════════════════════════
                    DADOS DO CLIENTE
═══════════════════════════════════════════════════════════════

Nome: Mariana Santos Oliveira
CPF: 987.654.321-00
Data de Nascimento: 10/05/1985
Profissão: Médica
Renda Mensal Comprovada: R$ 35.000,00

Endereço: Av. Paulista, 1000, Apto 121
Bairro: Bela Vista - São Paulo/SP
CEP: 01310-100

═══════════════════════════════════════════════════════════════
                    DADOS DO VEÍCULO
═══════════════════════════════════════════════════════════════

Marca/Modelo: BMW X5 xDrive40i
Ano/Modelo: 2024/2024
Cor: Preto Safira
Valor do Veículo: R$ 650.000,00
Valor de Entrada: R$ 200.000,00 (30,77%)

═══════════════════════════════════════════════════════════════
                    DADOS DA OPERAÇÃO
═══════════════════════════════════════════════════════════════

Valor Financiado: R$ 450.000,00
Prazo: 48 meses
Taxa de Juros: 1,29% ao mês (16,59% ao ano)
Valor da Parcela: R$ 12.856,78
CET: 18,92% ao ano

Garantia: Alienação fiduciária do próprio veículo
Seguro: Obrigatório - Cobertura total

═══════════════════════════════════════════════════════════════
                    ANÁLISE DE CRÉDITO
═══════════════════════════════════════════════════════════════

Score de Crédito: 920/1000
Comprometimento de Renda Atual: 15%
Comprometimento com esta operação: 36,7%
Comprometimento Total: 51,7%

ALERTA: Comprometimento acima do limite de 35%!

Histórico:
- Cliente desde 2018
- Financiamento anterior quitado (2020-2023)
- Sem atrasos registrados

═══════════════════════════════════════════════════════════════
                    PARECER
═══════════════════════════════════════════════════════════════

Embora o comprometimento de renda esteja acima do limite padrão,
a cliente possui:
- Score excelente (920)
- Entrada expressiva (30,77%)
- Histórico impecável
- Patrimônio declarado de R$ 2.500.000,00

Recomendação: APROVAR COM RESSALVAS
Exigências adicionais:
1. Seguro de proteção financeira
2. Débito automático obrigatório

Analista: Carlos Alberto
Matrícula: 78901
"""

with open("documentos/financiamento_veiculo.txt", "w", encoding="utf-8") as f:
    f.write(CONTRATO_EXEMPLO_2)

print("📄 Documento adicional criado: financiamento_veiculo.txt")

📄 Documento adicional criado: financiamento_veiculo.txt


---

## 🏗️ Parte 5: Projeto Prático - O Auditor de Contratos

Vamos construir um sistema completo que:
1. Recebe documentos de propostas/contratos
2. Extrai dados estruturados automaticamente
3. Valida contra regras de negócio
4. Gera um relatório de auditoria

In [19]:
# Schema completo para extração de propostas de crédito

from pydantic import BaseModel, Field, field_validator
from typing import Optional, List
from enum import Enum
from datetime import date

class TipoPessoa(str, Enum):
    PF = "PESSOA_FÍSICA"
    PJ = "PESSOA_JURÍDICA"

class TipoOperacao(str, Enum):
    CAPITAL_GIRO = "CAPITAL_DE_GIRO"
    FINANCIAMENTO_VEICULO = "FINANCIAMENTO_VEÍCULO"
    FINANCIAMENTO_IMOVEL = "FINANCIAMENTO_IMÓVEL"
    CREDITO_PESSOAL = "CRÉDITO_PESSOAL"
    ANTECIPACAO_RECEBIVEIS = "ANTECIPAÇÃO_RECEBÍVEIS"

class ScoreRisco(str, Enum):
    BAIXO = "BAIXO"
    MEDIO = "MÉDIO"
    ALTO = "ALTO"
    MUITO_ALTO = "MUITO_ALTO"

class Recomendacao(str, Enum):
    APROVAR = "APROVAR"
    APROVAR_COM_RESSALVAS = "APROVAR_COM_RESSALVAS"
    REVISAR = "REVISAR"
    REJEITAR = "REJEITAR"

class GarantiaExtraida(BaseModel):
    """Garantia extraída do documento."""
    tipo: str = Field(description="Tipo de garantia (alienação, hipoteca, aval, etc.)")
    descricao: str = Field(description="Descrição detalhada da garantia")
    valor: Optional[float] = Field(default=None, description="Valor da garantia em reais")

class NaoConformidade(BaseModel):
    """Não conformidade identificada."""
    codigo: str = Field(description="Código da não conformidade")
    severidade: str = Field(description="CRÍTICA, MAIOR, MENOR ou OBSERVAÇÃO")
    descricao: str = Field(description="Descrição do problema identificado")
    acao_requerida: str = Field(description="Ação necessária para correção")

class PropostaExtraida(BaseModel):
    """Schema completo para extração de proposta de crédito."""
    
    # Identificação
    numero_proposta: str = Field(description="Número/protocolo da proposta")
    data_proposta: str = Field(description="Data da proposta")
    tipo_pessoa: TipoPessoa = Field(description="Se é pessoa física ou jurídica")
    tipo_operacao: TipoOperacao = Field(description="Tipo da operação de crédito")
    
    # Dados do cliente
    nome_cliente: str = Field(description="Nome/Razão Social do cliente")
    documento_cliente: str = Field(description="CPF ou CNPJ do cliente")
    
    # Dados financeiros do cliente
    renda_faturamento: float = Field(description="Renda mensal (PF) ou faturamento anual (PJ)")
    score_credito: Optional[int] = Field(default=None, description="Score de crédito (0-1000)")
    
    # Dados da operação
    valor_solicitado: float = Field(description="Valor solicitado em reais")
    prazo_meses: int = Field(description="Prazo em meses")
    taxa_juros_mensal: float = Field(description="Taxa de juros mensal (%)")
    taxa_juros_anual: float = Field(description="Taxa de juros anual (%)")
    valor_parcela: float = Field(description="Valor de cada parcela")
    cet_anual: Optional[float] = Field(default=None, description="Custo Efetivo Total anual (%)")
    
    # Garantias
    garantias: List[GarantiaExtraida] = Field(description="Lista de garantias oferecidas")
    valor_total_garantias: float = Field(description="Soma de todas as garantias")
    cobertura_garantias: float = Field(description="Percentual de cobertura (garantias/valor)")
    
    # Análise de risco
    comprometimento_renda: Optional[float] = Field(default=None, description="% de comprometimento de renda")
    score_risco: ScoreRisco = Field(description="Score de risco calculado")
    
    # Não conformidades identificadas
    nao_conformidades: List[NaoConformidade] = Field(
        default_factory=list,
        description="Lista de não conformidades identificadas"
    )
    
    # Parecer final
    recomendacao: Recomendacao = Field(description="Recomendação de aprovação")
    justificativa_recomendacao: str = Field(description="Justificativa detalhada da recomendação")

print("✅ Schema PropostaExtraida definido!")
print(f"\n📋 Total de campos: {len(PropostaExtraida.model_fields)}")

✅ Schema PropostaExtraida definido!

📋 Total de campos: 22


In [20]:
# Classe do Auditor de Contratos

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from typing import List, Dict, Any
import json

class AuditorContratos:
    """
    Sistema de Auditoria de Contratos com RAG e Extração Estruturada.
    
    Funcionalidades:
    - Extração automática de dados de propostas/contratos
    - Validação contra políticas de compliance (RAG)
    - Identificação de não conformidades
    - Geração de parecer de auditoria
    """
    
    def __init__(self, vector_store: Chroma):
        self.vector_store = vector_store
        self.llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
        self.extrator = self.llm.with_structured_output(PropostaExtraida)
        self.retriever = vector_store.as_retriever(search_kwargs={"k": 4})
        self._setup_prompts()
    
    def _setup_prompts(self):
        """Configura os prompts do sistema."""
        
        self.prompt_extracao = ChatPromptTemplate.from_messages([
            ("system", """Você é um auditor especializado em análise de propostas de crédito.

Sua tarefa é extrair TODAS as informações do documento fornecido e avaliar a conformidade.

## REGRAS DE COMPLIANCE (use para identificar não conformidades):

{politicas}

## CRITÉRIOS DE AVALIAÇÃO DE RISCO:

BAIXO:
- Cobertura de garantias > 150%
- Score > 800
- Taxa de juros dentro da média de mercado
- Comprometimento de renda < 30%

MÉDIO:
- Cobertura de garantias entre 100-150%
- Score entre 700-800
- Comprometimento entre 30-40%

ALTO:
- Cobertura de garantias entre 80-100%
- Score entre 600-700
- Comprometimento entre 40-50%

MUITO_ALTO:
- Cobertura < 80%
- Score < 600
- Comprometimento > 50%
- Múltiplos fatores de risco

## IDENTIFICAÇÃO DE NÃO CONFORMIDADES:

- NC-001: Comprometimento de renda acima do limite (severidade: MAIOR)
- NC-002: Taxa de juros acima do permitido (severidade: CRÍTICA)
- NC-003: Garantias insuficientes (severidade: MAIOR)
- NC-004: Score abaixo do mínimo (severidade: CRÍTICA)
- NC-005: Documentação incompleta (severidade: MENOR)
- NC-006: CET não informado (severidade: CRÍTICA)

Seja preciso e criterioso na análise."""),
            ("human", "Analise esta proposta e extraia todos os dados:\n\n{documento}")
        ])
    
    def _buscar_politicas(self, tipo_operacao: str) -> str:
        """Busca políticas relevantes no vector store."""
        query = f"política de crédito {tipo_operacao} limites taxas garantias"
        docs = self.retriever.invoke(query)
        return "\n\n".join([doc.page_content for doc in docs])
    
    def auditar(self, documento: str) -> PropostaExtraida:
        """
        Realiza a auditoria completa de um documento.
        
        Args:
            documento: Texto do documento a ser auditado
            
        Returns:
            PropostaExtraida com todos os dados e análise
        """
        # Busca políticas relevantes
        politicas = self._buscar_politicas("pessoa física pessoa jurídica")
        
        # Cria a chain de extração
        chain = self.prompt_extracao | self.extrator
        
        # Executa a extração
        resultado = chain.invoke({
            "documento": documento,
            "politicas": politicas
        })
        
        return resultado
    
    def gerar_relatorio(self, proposta: PropostaExtraida) -> str:
        """
        Gera um relatório formatado da auditoria.
        """
        relatorio = []
        relatorio.append("="*70)
        relatorio.append("           RELATÓRIO DE AUDITORIA DE PROPOSTA DE CRÉDITO")
        relatorio.append("="*70)
        
        relatorio.append(f"\n📋 IDENTIFICAÇÃO")
        relatorio.append(f"   Proposta: {proposta.numero_proposta}")
        relatorio.append(f"   Data: {proposta.data_proposta}")
        relatorio.append(f"   Tipo: {proposta.tipo_operacao.value}")
        
        relatorio.append(f"\n👤 CLIENTE")
        relatorio.append(f"   Nome: {proposta.nome_cliente}")
        relatorio.append(f"   Documento: {proposta.documento_cliente}")
        relatorio.append(f"   Tipo: {proposta.tipo_pessoa.value}")
        if proposta.score_credito:
            relatorio.append(f"   Score: {proposta.score_credito}/1000")
        
        relatorio.append(f"\n💰 OPERAÇÃO")
        relatorio.append(f"   Valor Solicitado: R$ {proposta.valor_solicitado:,.2f}")
        relatorio.append(f"   Prazo: {proposta.prazo_meses} meses")
        relatorio.append(f"   Taxa Mensal: {proposta.taxa_juros_mensal}%")
        relatorio.append(f"   Taxa Anual: {proposta.taxa_juros_anual}%")
        relatorio.append(f"   Parcela: R$ {proposta.valor_parcela:,.2f}")
        if proposta.cet_anual:
            relatorio.append(f"   CET Anual: {proposta.cet_anual}%")
        
        relatorio.append(f"\n🛡️ GARANTIAS")
        for g in proposta.garantias:
            valor_str = f" - R$ {g.valor:,.2f}" if g.valor else ""
            relatorio.append(f"   • {g.tipo}: {g.descricao}{valor_str}")
        relatorio.append(f"   Total: R$ {proposta.valor_total_garantias:,.2f}")
        relatorio.append(f"   Cobertura: {proposta.cobertura_garantias:.1f}%")
        
        relatorio.append(f"\n⚠️ ANÁLISE DE RISCO")
        relatorio.append(f"   Score de Risco: {proposta.score_risco.value}")
        if proposta.comprometimento_renda:
            relatorio.append(f"   Comprometimento de Renda: {proposta.comprometimento_renda:.1f}%")
        
        if proposta.nao_conformidades:
            relatorio.append(f"\n❌ NÃO CONFORMIDADES IDENTIFICADAS")
            for nc in proposta.nao_conformidades:
                relatorio.append(f"   [{nc.severidade}] {nc.codigo}: {nc.descricao}")
                relatorio.append(f"      Ação: {nc.acao_requerida}")
        else:
            relatorio.append(f"\n✅ NENHUMA NÃO CONFORMIDADE IDENTIFICADA")
        
        relatorio.append(f"\n📝 PARECER FINAL")
        relatorio.append(f"   Recomendação: {proposta.recomendacao.value}")
        relatorio.append(f"   Justificativa: {proposta.justificativa_recomendacao}")
        
        relatorio.append("\n" + "="*70)
        
        return "\n".join(relatorio)

print("✅ Classe AuditorContratos definida!")

✅ Classe AuditorContratos definida!


In [21]:
# Instanciando o Auditor

auditor = AuditorContratos(vector_store=vector_store)

print("🔍 AUDITOR DE CONTRATOS")
print("=" * 60)
print("Sistema iniciado e pronto para auditar documentos!")

🔍 AUDITOR DE CONTRATOS
Sistema iniciado e pronto para auditar documentos!


In [22]:
# Auditando a proposta de crédito empresarial

print("🔍 AUDITORIA 1: Proposta de Crédito Empresarial")
print("=" * 60)

# Lê o documento
with open("documentos/proposta_credito.txt", "r", encoding="utf-8") as f:
    documento_pj = f.read()

# Executa a auditoria
resultado_pj = auditor.auditar(documento_pj)

# Gera e exibe o relatório
relatorio_pj = auditor.gerar_relatorio(resultado_pj)
print(relatorio_pj)

🔍 AUDITORIA 1: Proposta de Crédito Empresarial
           RELATÓRIO DE AUDITORIA DE PROPOSTA DE CRÉDITO

📋 IDENTIFICAÇÃO
   Proposta: PROP-2024-00547
   Data: 20/01/2024
   Tipo: CAPITAL_DE_GIRO

👤 CLIENTE
   Nome: Tech Solutions Ltda.
   Documento: 12.345.678/0001-90
   Tipo: PESSOA_JURÍDICA
   Score: 850/1000

💰 OPERAÇÃO
   Valor Solicitado: R$ 500,000.00
   Prazo: 24 meses
   Taxa Mensal: 1.89%
   Taxa Anual: 25.12%
   Parcela: R$ 25,847.33
   CET Anual: 28.45%

🛡️ GARANTIAS
   • Alienação Fiduciária: Servidores e equipamentos de TI - R$ 180,000.00
   • Recebíveis: Contratos de prestação de serviços - R$ 250,000.00
   • Aval dos Sócios: Roberto Almeida - Patrimônio declarado: R$ 800.000,00; Fernanda Costa - Patrimônio declarado: R$ 450.000,00 - R$ 1,250,000.00
   Total: R$ 1,680,000.00
   Cobertura: 336.0%

⚠️ ANÁLISE DE RISCO
   Score de Risco: BAIXO

✅ NENHUMA NÃO CONFORMIDADE IDENTIFICADA

📝 PARECER FINAL
   Recomendação: APROVAR
   Justificativa: Cliente com bom histórico e rela

In [23]:
# Auditando a proposta de financiamento de veículo

print("🔍 AUDITORIA 2: Financiamento de Veículo")
print("=" * 60)

# Lê o documento
with open("documentos/financiamento_veiculo.txt", "r", encoding="utf-8") as f:
    documento_veiculo = f.read()

# Executa a auditoria
resultado_veiculo = auditor.auditar(documento_veiculo)

# Gera e exibe o relatório
relatorio_veiculo = auditor.gerar_relatorio(resultado_veiculo)
print(relatorio_veiculo)

🔍 AUDITORIA 2: Financiamento de Veículo
           RELATÓRIO DE AUDITORIA DE PROPOSTA DE CRÉDITO

📋 IDENTIFICAÇÃO
   Proposta: FIN-VEI-2024-01234
   Data: 22/01/2024
   Tipo: FINANCIAMENTO_VEÍCULO

👤 CLIENTE
   Nome: Mariana Santos Oliveira
   Documento: 987.654.321-00
   Tipo: PESSOA_FÍSICA
   Score: 920/1000

💰 OPERAÇÃO
   Valor Solicitado: R$ 450,000.00
   Prazo: 48 meses
   Taxa Mensal: 1.29%
   Taxa Anual: 16.59%
   Parcela: R$ 12,856.78
   CET Anual: 18.92%

🛡️ GARANTIAS
   • alienação fiduciária: Alienação fiduciária do próprio veículo - R$ 650,000.00
   Total: R$ 650,000.00
   Cobertura: 100.0%

⚠️ ANÁLISE DE RISCO
   Score de Risco: ALTO
   Comprometimento de Renda: 36.7%

❌ NÃO CONFORMIDADES IDENTIFICADAS
   [MAIOR] NC-001: Comprometimento de renda acima do limite (36,7%)
      Ação: Revisar comprometimento de renda e considerar a aprovação com ressalvas.

📝 PARECER FINAL
   Recomendação: APROVAR_COM_RESSALVAS
   Justificativa: Embora o comprometimento de renda esteja acima d

In [24]:
# Exportando resultados como JSON

print("📤 EXPORTANDO RESULTADOS")
print("=" * 60)

# Exporta para JSON
resultado_json_pj = resultado_pj.model_dump_json(indent=2)
resultado_json_veiculo = resultado_veiculo.model_dump_json(indent=2)

# Salva em arquivos
with open("documentos/auditoria_pj.json", "w", encoding="utf-8") as f:
    f.write(resultado_json_pj)

with open("documentos/auditoria_veiculo.json", "w", encoding="utf-8") as f:
    f.write(resultado_json_veiculo)

print("\n✅ Arquivos exportados:")
print("   • documentos/auditoria_pj.json")
print("   • documentos/auditoria_veiculo.json")

print("\n📄 Preview do JSON (Proposta PJ):")
print(resultado_json_pj[:1000] + "...")

📤 EXPORTANDO RESULTADOS

✅ Arquivos exportados:
   • documentos/auditoria_pj.json
   • documentos/auditoria_veiculo.json

📄 Preview do JSON (Proposta PJ):
{
  "numero_proposta": "PROP-2024-00547",
  "data_proposta": "20/01/2024",
  "tipo_pessoa": "PESSOA_JURÍDICA",
  "tipo_operacao": "CAPITAL_DE_GIRO",
  "nome_cliente": "Tech Solutions Ltda.",
  "documento_cliente": "12.345.678/0001-90",
  "renda_faturamento": 3500000.0,
  "score_credito": 850,
  "valor_solicitado": 500000.0,
  "prazo_meses": 24,
  "taxa_juros_mensal": 1.89,
  "taxa_juros_anual": 25.12,
  "valor_parcela": 25847.33,
  "cet_anual": 28.45,
  "garantias": [
    {
      "tipo": "Alienação Fiduciária",
      "descricao": "Servidores e equipamentos de TI",
      "valor": 180000.0
    },
    {
      "tipo": "Recebíveis",
      "descricao": "Contratos de prestação de serviços",
      "valor": 250000.0
    },
    {
      "tipo": "Aval dos Sócios",
      "descricao": "Roberto Almeida - Patrimônio declarado: R$ 800.000,00; Fernand

---

## 📚 Parte 6: Validação com Regras de Negócio

Vamos adicionar validação automática usando Pydantic validators.

In [25]:
# Classe de validação com regras de negócio

from pydantic import BaseModel, Field, model_validator
from typing import List, Optional

class ValidacaoProposta(BaseModel):
    """Validação de proposta com regras de negócio."""
    
    valor_solicitado: float
    prazo_meses: int
    taxa_juros_mensal: float
    valor_total_garantias: float
    score_credito: Optional[int] = None
    comprometimento_renda: Optional[float] = None
    
    # Resultados da validação
    validacoes: List[str] = Field(default_factory=list)
    alertas: List[str] = Field(default_factory=list)
    bloqueios: List[str] = Field(default_factory=list)
    
    @model_validator(mode='after')
    def validar_regras_negocio(self):
        """Valida todas as regras de negócio."""
        
        # Regra 1: Cobertura de garantias
        cobertura = (self.valor_total_garantias / self.valor_solicitado) * 100
        if cobertura >= 100:
            self.validacoes.append(f"✅ Cobertura de garantias adequada: {cobertura:.1f}%")
        elif cobertura >= 80:
            self.alertas.append(f"⚠️ Cobertura de garantias marginal: {cobertura:.1f}%")
        else:
            self.bloqueios.append(f"❌ Cobertura de garantias insuficiente: {cobertura:.1f}%")
        
        # Regra 2: Taxa de juros
        if self.taxa_juros_mensal <= 2.5:
            self.validacoes.append(f"✅ Taxa de juros dentro do limite: {self.taxa_juros_mensal}% a.m.")
        elif self.taxa_juros_mensal <= 3.5:
            self.alertas.append(f"⚠️ Taxa de juros acima da média: {self.taxa_juros_mensal}% a.m.")
        else:
            self.bloqueios.append(f"❌ Taxa de juros excessiva: {self.taxa_juros_mensal}% a.m.")
        
        # Regra 3: Prazo
        if self.prazo_meses <= 60:
            self.validacoes.append(f"✅ Prazo dentro do limite: {self.prazo_meses} meses")
        else:
            self.alertas.append(f"⚠️ Prazo longo: {self.prazo_meses} meses - requer aprovação especial")
        
        # Regra 4: Score de crédito
        if self.score_credito:
            if self.score_credito >= 700:
                self.validacoes.append(f"✅ Score de crédito bom: {self.score_credito}")
            elif self.score_credito >= 600:
                self.alertas.append(f"⚠️ Score de crédito médio: {self.score_credito}")
            else:
                self.bloqueios.append(f"❌ Score de crédito abaixo do mínimo: {self.score_credito}")
        
        # Regra 5: Comprometimento de renda
        if self.comprometimento_renda:
            if self.comprometimento_renda <= 35:
                self.validacoes.append(f"✅ Comprometimento de renda adequado: {self.comprometimento_renda}%")
            elif self.comprometimento_renda <= 50:
                self.alertas.append(f"⚠️ Comprometimento de renda elevado: {self.comprometimento_renda}%")
            else:
                self.bloqueios.append(f"❌ Comprometimento de renda excessivo: {self.comprometimento_renda}%")
        
        return self
    
    def pode_aprovar(self) -> bool:
        """Retorna True se não houver bloqueios."""
        return len(self.bloqueios) == 0
    
    def gerar_parecer(self) -> str:
        """Gera parecer da validação."""
        linhas = ["\n🔍 VALIDAÇÃO DE REGRAS DE NEGÓCIO", "="*50]
        
        if self.validacoes:
            linhas.append("\n✅ APROVAÇÕES:")
            linhas.extend([f"   {v}" for v in self.validacoes])
        
        if self.alertas:
            linhas.append("\n⚠️ ALERTAS:")
            linhas.extend([f"   {a}" for a in self.alertas])
        
        if self.bloqueios:
            linhas.append("\n❌ BLOQUEIOS:")
            linhas.extend([f"   {b}" for b in self.bloqueios])
        
        linhas.append("\n" + "="*50)
        if self.pode_aprovar():
            if self.alertas:
                linhas.append("📋 PARECER: APROVAR COM RESSALVAS")
            else:
                linhas.append("📋 PARECER: APROVAR")
        else:
            linhas.append("📋 PARECER: NECESSITA ANÁLISE MANUAL")
        
        return "\n".join(linhas)

print("✅ Classe ValidacaoProposta definida!")

✅ Classe ValidacaoProposta definida!


In [26]:
# Testando a validação com os resultados das auditorias

print("📊 VALIDAÇÃO DA PROPOSTA PJ (Tech Solutions)")
print("=" * 60)

validacao_pj = ValidacaoProposta(
    valor_solicitado=resultado_pj.valor_solicitado,
    prazo_meses=resultado_pj.prazo_meses,
    taxa_juros_mensal=resultado_pj.taxa_juros_mensal,
    valor_total_garantias=resultado_pj.valor_total_garantias,
    score_credito=resultado_pj.score_credito
)

print(validacao_pj.gerar_parecer())

📊 VALIDAÇÃO DA PROPOSTA PJ (Tech Solutions)

🔍 VALIDAÇÃO DE REGRAS DE NEGÓCIO

✅ APROVAÇÕES:
   ✅ Cobertura de garantias adequada: 336.0%
   ✅ Taxa de juros dentro do limite: 1.89% a.m.
   ✅ Prazo dentro do limite: 24 meses
   ✅ Score de crédito bom: 850

📋 PARECER: APROVAR


In [27]:
# Validação do financiamento de veículo

print("\n📊 VALIDAÇÃO DO FINANCIAMENTO DE VEÍCULO (Mariana)")
print("=" * 60)

validacao_veiculo = ValidacaoProposta(
    valor_solicitado=resultado_veiculo.valor_solicitado,
    prazo_meses=resultado_veiculo.prazo_meses,
    taxa_juros_mensal=resultado_veiculo.taxa_juros_mensal,
    valor_total_garantias=resultado_veiculo.valor_total_garantias,
    score_credito=resultado_veiculo.score_credito,
    comprometimento_renda=resultado_veiculo.comprometimento_renda
)

print(validacao_veiculo.gerar_parecer())


📊 VALIDAÇÃO DO FINANCIAMENTO DE VEÍCULO (Mariana)

🔍 VALIDAÇÃO DE REGRAS DE NEGÓCIO

✅ APROVAÇÕES:
   ✅ Cobertura de garantias adequada: 144.4%
   ✅ Taxa de juros dentro do limite: 1.29% a.m.
   ✅ Prazo dentro do limite: 48 meses
   ✅ Score de crédito bom: 920

⚠️ ALERTAS:
   ⚠️ Comprometimento de renda elevado: 36.7%

📋 PARECER: APROVAR COM RESSALVAS


---

## 📚 Parte 7: Sistema Integrado Final

Vamos consolidar tudo em uma classe que pode ser reutilizada em produção.

In [49]:
# Sistema Integrado de Auditoria com RAG

from typing import Tuple
from datetime import datetime

class SistemaAuditoriaRAG:
    """
    Sistema completo de auditoria de contratos com:
    - Extração estruturada de dados
    - Validação RAG contra políticas
    - Validação de regras de negócio
    - Geração de relatórios
    """
    
    def __init__(self, documentos_politicas: List[dict]):
        """Inicializa o sistema com documentos de políticas."""
        print("🔄 Inicializando Sistema de Auditoria RAG...")
        
        # Configura embeddings
        self.embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
        
        # Configura LLM para consultas
        self.llm = ChatOpenAI(model="gpt-4o", temperature=0)
        
        # Processa documentos de políticas
        self._carregar_politicas(documentos_politicas)
        
        # Configura o auditor
        self.auditor = AuditorContratos(vector_store=self.vector_store)
        
        # Configura RAG chain para consultas
        self._configurar_rag_chain()
        
        print("✅ Sistema inicializado com sucesso!")
    
    def _carregar_politicas(self, documentos: List[dict]):
        """Carrega e indexa documentos de políticas."""
        from langchain_text_splitters import RecursiveCharacterTextSplitter
        from langchain_core.documents import Document
        
        splitter = RecursiveCharacterTextSplitter(
            chunk_size=500,
            chunk_overlap=50
        )
        
        docs_processados = []
        for doc in documentos:
            chunks = splitter.split_text(doc["conteudo"])
            for i, chunk in enumerate(chunks):
                docs_processados.append(Document(
                    page_content=chunk,
                    metadata={"titulo": doc["titulo"], "chunk": i}
                ))
        
        self.vector_store = Chroma.from_documents(
            documents=docs_processados,
            embedding=self.embeddings,
            collection_name="politicas_compliance"
        )
        
        print(f"   📚 {len(documentos)} documentos carregados ({len(docs_processados)} chunks)")
    
    def _configurar_rag_chain(self):
        """Configura a chain RAG para consultas às políticas."""
        from langchain_core.prompts import ChatPromptTemplate
        from langchain_core.runnables import RunnablePassthrough
        from langchain_core.output_parsers import StrOutputParser
        
        self.retriever = self.vector_store.as_retriever(search_kwargs={"k": 4})
        
        prompt = ChatPromptTemplate.from_messages([
            ("system", """Você é um especialista em compliance bancário.
Use APENAS as informações fornecidas no contexto para responder.
Se a informação não estiver no contexto, diga que não encontrou nos documentos disponíveis.
Seja conciso e direto na resposta.

Contexto dos documentos:
{context}"""),
            ("human", "{question}")
        ])
        
        def format_docs(docs):
            return "\n\n".join([f"[{doc.metadata['titulo']}]\n{doc.page_content}" for doc in docs])
        
        self.rag_chain = (
            {"context": self.retriever | format_docs, "question": RunnablePassthrough()}
            | prompt
            | self.llm
            | StrOutputParser()
        )
    
    def consultar_politica(self, pergunta: str) -> str:
        """
        Consulta as políticas de compliance usando RAG.
        
        Args:
            pergunta: Pergunta sobre as políticas
            
        Returns:
            Resposta baseada nos documentos de política
        """
        return self.rag_chain.invoke(pergunta)
    
    def processar_proposta(self, texto_proposta: str) -> Tuple[PropostaExtraida, ValidacaoProposta, str]:
        """
        Processa uma proposta completa.
        
        Returns:
            Tuple com (dados extraídos, validação, relatório)
        """
        # 1. Extrai dados
        proposta = self.auditor.auditar(texto_proposta)
        
        # 2. Calcula valor total das garantias
        valor_total_garantias = 0.0
        if proposta.garantias:
            valor_total_garantias = sum(g.valor for g in proposta.garantias if g.valor)
        
        # 3. Valida regras de negócio
        validacao = ValidacaoProposta(
            valor_solicitado=proposta.valor_solicitado,
            prazo_meses=proposta.prazo_meses,
            taxa_juros_mensal=proposta.taxa_juros_mensal,
            valor_total_garantias=valor_total_garantias,
            comprometimento_renda=proposta.comprometimento_renda,
            score_credito=proposta.score_credito
        )
        
        # 4. Gera relatório
        relatorio = self._gerar_relatorio(proposta, validacao)
        
        return proposta, validacao, relatorio
    
    def _classificar_risco(self, validacao: ValidacaoProposta) -> str:
        """Classifica o nível de risco baseado na validação."""
        if validacao.bloqueios:
            return "ALTO"
        elif validacao.alertas:
            return "MÉDIO"
        else:
            return "BAIXO"
    
    def _gerar_relatorio(self, proposta: PropostaExtraida, validacao: ValidacaoProposta) -> str:
        """Gera relatório completo de auditoria."""
        # Usa RAG para buscar políticas relevantes
        retriever = self.vector_store.as_retriever(search_kwargs={"k": 3})
        
        contexto = f"""
        Tipo de Operação: {proposta.tipo_operacao}
        Valor: R$ {proposta.valor_solicitado:,.2f}
        Tipo de Cliente: {proposta.tipo_pessoa}
        """
        
        docs = retriever.invoke(contexto)
        politicas = "\n".join([d.page_content for d in docs])
        
        # Classifica o risco
        nivel_risco = self._classificar_risco(validacao)
        
        # Trata valores opcionais
        score_str = f"{proposta.score_credito}/1000" if proposta.score_credito else "N/A"
        comprometimento_str = f"{proposta.comprometimento_renda:.1f}%" if proposta.comprometimento_renda else "N/A"
        
        relatorio = f"""
╔══════════════════════════════════════════════════════════════════╗
║              RELATÓRIO DE AUDITORIA - SISTEMA RAG                 ║
╠══════════════════════════════════════════════════════════════════╣
║ Data: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}                                      ║
║ Protocolo: {proposta.numero_proposta or 'N/A':12}                                    ║
╚══════════════════════════════════════════════════════════════════╝

📋 DADOS DA PROPOSTA
{'='*60}
• Cliente: {proposta.nome_cliente}
• Tipo: {proposta.tipo_pessoa.value if proposta.tipo_pessoa else 'N/A'}
• Operação: {proposta.tipo_operacao.value if proposta.tipo_operacao else 'N/A'}
• Valor: R$ {proposta.valor_solicitado:,.2f}
• Prazo: {proposta.prazo_meses} meses
• Taxa: {proposta.taxa_juros_mensal:.2f}% a.m.

📊 ANÁLISE DE RISCO
{'='*60}
• Score de Crédito: {score_str}
• Comprometimento de Renda: {comprometimento_str}
• Classificação de Risco: {nivel_risco}

✅ VALIDAÇÕES
{'='*60}
{validacao.gerar_parecer()}

📚 POLÍTICAS APLICÁVEIS
{'='*60}
{politicas[:500]}...

{'='*60}
                    FIM DO RELATÓRIO
{'='*60}
"""
        return relatorio
    
    def processar_lote(self, propostas: List[str]) -> List[dict]:
        """Processa um lote de propostas."""
        resultados = []
        for i, proposta in enumerate(propostas, 1):
            print(f"\n📄 Processando proposta {i}/{len(propostas)}...")
            dados, validacao, relatorio = self.processar_proposta(proposta)
            resultados.append({
                "proposta": dados,
                "validacao": validacao,
                "relatorio": relatorio
            })
        return resultados

print("✅ Classe SistemaAuditoriaRAG definida!")

✅ Classe SistemaAuditoriaRAG definida!


In [50]:
# Testando o Sistema Integrado

print("🚀 INICIALIZANDO SISTEMA DE AUDITORIA INTEGRADO")
print("=" * 60)

# Cria o sistema
sistema = SistemaAuditoriaRAG(documentos_politicas=DOCUMENTOS_COMPLIANCE)

print("\n" + "="*60)

🚀 INICIALIZANDO SISTEMA DE AUDITORIA INTEGRADO
🔄 Inicializando Sistema de Auditoria RAG...
   📚 4 documentos carregados (12 chunks)
✅ Sistema inicializado com sucesso!



In [51]:
# Processando a proposta empresarial pelo sistema integrado

print("\n📋 PROCESSANDO PROPOSTA: Tech Solutions Ltda.")
print("="*60)

proposta_extraida, validacao_regras, relatorio_final = sistema.processar_proposta(PROPOSTA_CREDITO_TEXTO)

print(relatorio_final)


📋 PROCESSANDO PROPOSTA: Tech Solutions Ltda.

╔══════════════════════════════════════════════════════════════════╗
║              RELATÓRIO DE AUDITORIA - SISTEMA RAG                 ║
╠══════════════════════════════════════════════════════════════════╣
║ Data: 28/01/2026 13:19:12                                      ║
║ Protocolo: PROP-2024-00547                                    ║
╚══════════════════════════════════════════════════════════════════╝

📋 DADOS DA PROPOSTA
• Cliente: Tech Solutions Ltda.
• Tipo: PESSOA_JURÍDICA
• Operação: CAPITAL_DE_GIRO
• Valor: R$ 500,000.00
• Prazo: 24 meses
• Taxa: 1.89% a.m.

📊 ANÁLISE DE RISCO
• Score de Crédito: 850/1000
• Comprometimento de Renda: N/A
• Classificação de Risco: BAIXO

✅ VALIDAÇÕES

🔍 VALIDAÇÃO DE REGRAS DE NEGÓCIO

✅ APROVAÇÕES:
   ✅ Cobertura de garantias adequada: 336.0%
   ✅ Taxa de juros dentro do limite: 1.89% a.m.
   ✅ Prazo dentro do limite: 24 meses
   ✅ Score de crédito bom: 850

📋 PARECER: APROVAR

📚 POLÍTICAS APLICÁV

In [48]:
# Consultando políticas via RAG

print("\n🔍 CONSULTAS ÀS POLÍTICAS (RAG)")
print("="*60)

perguntas = [
    "Qual o limite máximo de financiamento para capital de giro PJ?",
    "Quais são os critérios de elegibilidade para crédito PJ?",
    "O que caracteriza uma não conformidade crítica em auditoria?"
]

for p in perguntas:
    print(f"\n❓ {p}")
    resposta = sistema.consultar_politica(p)
    print(f"\n💬 {resposta}")
    print("-" * 60)


🔍 CONSULTAS ÀS POLÍTICAS (RAG)

❓ Qual o limite máximo de financiamento para capital de giro PJ?

💬 O limite máximo de financiamento para capital de giro PJ é até 50% do faturamento mensal médio.
------------------------------------------------------------

❓ Quais são os critérios de elegibilidade para crédito PJ?

💬 Não encontrei nos documentos disponíveis informações sobre os critérios de elegibilidade para crédito PJ.
------------------------------------------------------------

❓ O que caracteriza uma não conformidade crítica em auditoria?

💬 Uma não conformidade crítica em auditoria é caracterizada por requerer correção imediata e reporte à diretoria.
------------------------------------------------------------


---

## 📝 Resumo da Aula 2

### O que aprendemos:

1. **Chat Genérico vs Extração Estruturada**
   - Chat genérico retorna texto livre, difícil de processar
   - Extração estruturada usa Pydantic para garantir formato
   - `with_structured_output()` força o LLM a seguir um schema

2. **RAG (Retrieval-Augmented Generation)**
   - Permite que o LLM use conhecimento de documentos internos
   - Componentes: Embeddings → Vector Store → Retriever → LLM
   - ChromaDB é uma opção leve para Vector Store

3. **Embeddings e Similaridade**
   - Embeddings convertem texto em vetores numéricos
   - Similaridade de cosseno mede proximidade semântica
   - Chunking divide documentos grandes em pedaços gerenciáveis

4. **Function Calling e Output Parsers**
   - Pydantic valida e estrutura saídas do LLM
   - Validators permitem regras de negócio customizadas
   - JSON export facilita integração com sistemas

5. **Projeto: O Auditor de Contratos**
   - Sistema completo de extração e validação
   - Integração com base de políticas via RAG
   - Geração automática de relatórios de auditoria

---

### 📚 Próxima Aula: Orquestração de Estado com LangGraph

Na Aula 3, você aprenderá:
- Grafos Computacionais: Nós e Arestas
- Gerenciamento de Estado Global
- Persistência com Checkpoints
- Human-in-the-loop para aprovações
- Projeto: O Motor de Decisão

---

**Parabéns por completar a Aula 2!** 🎉